In [1]:
from API_webHMI import *
from defs import *
from head import headers, device_adress
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure



ModuleNotFoundError: No module named 'requests'

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
def conn():
    print('1: Connection Req\n')
    # displayHeader(headers)
    req1 = connectionList(device_adress, headers)  # pobranie listy polaczen plc.
    return req1

In [ ]:
req1=conn()
connections=pd.DataFrame(req1)
connections['plcid']=connections['id']
connections.index=connections['id']
connections.head()

In [ ]:
def reg():
    print('\n2 :Registers Req\n')
    # displayHeader(headers)  # wystarczy podstawowy naglowek
    req2 = registerList(device_adress, headers)  # odczytanie listy rejestrow
    return req2
registers=pd.DataFrame(reg())
registers.head()

In [ ]:
regList = pd.merge(registers,connections[['title','plcid']],on='plcid')
regList.index=regList['id']
regList.head()

In [ ]:
regList[regList['plcid']=='3']

In [ ]:
regListEnable=regList[regList['disabled']=='0']
regListEnable.head()

In [ ]:
regList[regList['title_x'].str.contains('Flow m3/h')].head()

In [ ]:
def graphListReq():
    print('\n3 :Graph Req\n')
    # displayHeader(headers)  # wystarczy podstawowy naglowek
    req3 = graphList(device_adress, headers)
    return req3


In [ ]:
graph=graphListReq()

In [ ]:
graphList=pd.DataFrame(graph)
graphList.index=graphList['id']
graphList

In [ ]:
from datetime import datetime

def graphDataReq(headers, k):
    print('\n4 :Graph Data Req\n')
    # displayHeader(headers)  # wystarczy podstawowy naglowek
    req4 = getGraph(device_adress, headers, k)  # odczytanie danych z wykresow
    return req4



def range(rok,miesiac,dzien):
    wh_slices = 800
    length = 7
    dt = datetime(rok,miesiac,dzien)
    unixtime = time.mktime(dt.timetuple())  # - time.timezone
    date = datetime.fromtimestamp(unixtime).strftime('%Y-%m-%d  %H:%M:%S')
    wh_start = int(unixtime)
    return wh_start, wh_slices, date, length

def head(wh_start, wh_slices, lenght):
    # Ustalenie nagłowka dla wykresu
    wh_stop = wh_start + lenght * 60 * 60 * 24
    headers['X-WH-CONNS'] = ''
    headers['X-WH-REGS'] = ''
    headers['X-WH-START'] = str(wh_start)
    headers['X-WH-END'] = str(wh_stop)
    headers['X-WH-SLICES'] = str(wh_slices)
    return headers

wh_start, wh_slices, date, lenght=range(2019,2,4)
headers = head(wh_start, wh_slices, lenght)

headers

In [ ]:
rawData=graphDataReq(headers,18)

In [ ]:
raw=pd.DataFrame(rawData)
wiatr=raw
wiatr['x'] = pd.to_datetime(wiatr['x'], unit='ms')
wiatr.size

In [ ]:
old_names = wiatr.columns.tolist()
new_names = [(i,regList.loc[i,'title_y'],regList.loc[i,'title_x']) for i in old_names if i !='x']
wiatr.rename(columns=dict(zip(old_names, new_names)), inplace=True)
wiatr.head()

In [ ]:
wind=pd.DataFrame(dict([(('Time',''),wiatr['x'])]))
dd=[wind]
# dd=[]
for i in wiatr.keys():
    if i!='x':
        vals=['min','avg','max']
        devs=wiatr[i].str.split(';', expand=True).rename(columns = lambda x:vals[x] )  
        dfp= pd.DataFrame(dict([
            ((i,'min'), devs['min'].astype('float')),
            ((i,'avg'), devs['avg'].astype('float')),
            ((i,'max'), devs['max'].astype('float')),
        ]))
        dd.append(dfp)
df = pd.concat(dd, axis=1)
df.set_index('Time', inplace=True)
# df.index=wiatr['x']
# df=df.reset_index()
# pd.merge(left=df, right=wind, left_index=True, right_index=True)
df.head()

In [ ]:
lista=[x for x in df.columns.tolist() if x[1]=='avg']
mask=df[lista]
mask.head()


In [ ]:
# plt.rcParams["figure.figsize"] = (20,10)
mask.plot(figsize=(20,8))
plt.title(new_names[0][2])
plt.xlabel("Data")
plt.ylabel(new_names[0][2])
plt.grid(True)
plt.legend(loc='center left', bbox_to_anchor=(1.03, 0.5))
plt.show()

### Agregacja statystyk

In [ ]:
mask.agg(['mean', 'min','max'])